<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_2/02_hopfield_network_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythonのクラスでホップフィールドネットワークを実装する
ホップフィールドネットワークのコードがどのように動作するのか、さらに詳しく解説します。

## ホップフィールドネットワークの全体コード
以降、クラス`HopfieldNetwork`を構成する各メソッドを解説します。

```python
import numpy as np

class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons))

    def train(self, patterns):
        for p in patterns:
            p = p.reshape(self.num_neurons, 1)  # パターンを列ベクトルに変換
            self.weights += np.dot(p, p.T)      # 外積で重み行列を更新
        np.fill_diagonal(self.weights, 0)       # 対角要素を0に設定

    def recall(self, pattern, steps=5):
        result = pattern.copy()
        for _ in range(steps):
            result = np.sign(np.dot(self.weights, result))
        return result
```

---

## 1. `__init__` メソッド（初期化）

```python
def __init__(self, num_neurons):
    self.num_neurons = num_neurons
    self.weights = np.zeros((num_neurons, num_neurons))
```

### 解説
- **役割**:
  - ネットワークの初期化を行います。
  - ニューロンの数と重み行列（weights）を初期化します。

- **パラメータ**:
  - `num_neurons`: ニューロンの数（ネットワークに記憶させるパターンの要素数に対応）。

- **動作**:
  - `self.num_neurons`: ネットワークのニューロン数を設定。
  - `self.weights`: **重み行列**（`num_neurons x num_neurons` の2次元配列）をゼロ行列で初期化。
    - この行列は、ニューロン同士の結合の強さを表します。
    - 初期状態では、すべての結合がゼロ（つまり影響を与えない）になっています。

---

## 2. `train` メソッド（学習）

```python
def train(self, patterns):
    for p in patterns:
        p = p.reshape(self.num_neurons, 1)  # パターンを列ベクトルに変換
        self.weights += np.dot(p, p.T)      # 外積で重み行列を更新
    np.fill_diagonal(self.weights, 0)       # 対角要素を0に設定
```

### 解説
- **役割**:
  - 複数のパターンをホップフィールドネットワークに**学習**させます。
  - 各パターンの外積（※補足で解説）を用いて、重み行列を更新します（**Hebbian学習**）。

- **パラメータ**:
  - `patterns`: 学習させたいパターンのリスト（各パターンは1次元の配列）。

- **動作の流れ**:
  1. **パターンを列ベクトルに変換**:
     ```python
     p = p.reshape(self.num_neurons, 1)
     ```
     - ここでは、パターン `p` を縦に並んだ**列ベクトル**に変換します。
     - 例: `[1, -1, 1]` → `[[1], [-1], [1]]`

  2. **重み行列の更新**:
     ```python
     self.weights += np.dot(p, p.T)
     ```
     - `np.dot(p, p.T)` は **外積** （※補足で解説）を計算します。
     - 外積を使って、ニューロン同士の結合の強さ（重み）を決定します。
     - 外積の例:
       - パターン `p = [[1], [-1], [1]]` の場合、`np.dot(p, p.T)` の結果は次のようになります：
         ```
         [[ 1, -1,  1],
          [-1,  1, -1],
          [ 1, -1,  1]]
         ```
     - 各パターンについて、この重み行列を累積していきます。

  3. **対角要素をゼロに設定**:
     ```python
     np.fill_diagonal(self.weights, 0)
     ```
     - 重み行列の **対角要素**（自己結合）はゼロに設定します。
     - これにより、各ニューロンが **自分自身に影響を与えない** ようにします。

---

## 3. `recall` メソッド（パターンの再現）

```python
def recall(self, pattern, steps=5):
    result = pattern.copy()
    for _ in range(steps):
        result = np.sign(np.dot(self.weights, result))
    return result
```

### 解説
- **役割**:
  - **ノイズの入ったパターン**を入力し、ホップフィールドネットワークが学習したパターンを**再現**（復元）する。
  - 反復的に重み行列を用いて入力を修正し、学習済みパターンに収束させます。

- **パラメータ**:
  - `pattern`: 再現したいパターン（入力として与える1次元配列）。
  - `steps`: 再現のために行う**更新の回数**（デフォルトは5回）。

- **動作の流れ**:
  1. **入力パターンのコピーを作成**:
     ```python
     result = pattern.copy()
     ```
     - 元の入力パターンを変更しないように、コピーを作成します。

  2. **重み行列を用いた更新ループ**:
     ```python
     for _ in range(steps):
         result = np.sign(np.dot(self.weights, result))
     ```
     - 重み行列と入力パターンの **内積** を計算し、その結果を符号関数（`np.sign`）を使って -1 または 1 に変換します。
     - これを `steps` 回繰り返して、**パターンが安定するまで更新** します。
       - 正の値 → 1
       - 負の値 → -1

  3. **修正されたパターンを返す**:
     - 更新が終了したら、修正されたパターンを返します。
     - このパターンは、ネットワークが学習したパターンに収束していることが期待されます。

---

### 具体例での動作の理解

- **学習したパターン**: `pattern1 = [1, -1, 1, -1, 1, -1, 1, -1, 1]`
- **ノイズの入った入力**: `noisy_pattern = [1, 1, 1, -1, 1, -1, 1, -1, -1]`
- **再現結果**:
  - ノイズの入った `noisy_pattern` を `recall` メソッドに入力すると、ホップフィールドネットワークは学習済みの `pattern1` に収束します。

---

### まとめ
- `train` メソッドでパターンを記憶し、重み行列を構築します。
- `recall` メソッドで、ノイズの入った入力を修正し、学習済みのパターンに収束させます。
- ホップフィールドネットワークは、**ノイズ除去** や **パターンの連想記憶** に適しており、シンプルな構造で強力な連想メモリを実現できます。

このクラスの基本的な仕組みを理解することで、ホップフィールドネットワークを使った応用タスク（画像のノイズ除去、パターンの復元など）に発展させることができます。

## （補足）「外積」についての解説

ホップフィールドネットワークの **`train` メソッド** の中で使用されている「外積」 (`np.dot(p, p.T)`) について、詳しく解説します。

---

### 1. **外積とは？**
外積（Outer Product）は、**ベクトルとベクトルから行列を生成する操作** のことを指します。具体的には、**縦ベクトル** と **横ベクトル** を掛け合わせることで、**行列** を作ります。

- **内積**（Dot Product）はスカラー（数値）を返すのに対して、
- **外積** は行列を返します。

#### 数学的な定義
2つのベクトル $\mathbf{a}$ と $\mathbf{b}$ の外積は、次のように表されます：
$$
\mathbf{a} \otimes \mathbf{b} = \mathbf{a} \cdot \mathbf{b}^T
$$
- ここで、$\mathbf{a}$ は **縦ベクトル**、$\mathbf{b}^T$ は **横ベクトル** にしたものです。
- 結果として、**行列** が得られます。

---

### 2. **ホップフィールドネットワークにおける外積の使い方**
ホップフィールドネットワークでは、**パターンの記憶** に外積を利用しています。この操作により、各ニューロン間の結合強度（重み行列）が決まります。

---

### 3. **具体例で理解する**

#### 例
ここでは、シンプルなパターン $p$ を使って、外積がどのように計算されるかを示します。

- **パターン** $p = [1, -1, 1]$
- **列ベクトル** として表すと：

$$
p = \begin{bmatrix} 1 \\ -1 \\ 1 \end{bmatrix}
$$

##### 外積の計算
$$
\text{外積: } p \otimes p^T = \begin{bmatrix} 1 \\ -1 \\ 1 \end{bmatrix} \times \begin{bmatrix} 1 & -1 & 1 \end{bmatrix}
$$

- 計算の結果は次の **行列** になります：

$$
\begin{bmatrix}
1 \times 1 & 1 \times -1 & 1 \times 1 \\
-1 \times 1 & -1 \times -1 & -1 \times 1 \\
1 \times 1 & 1 \times -1 & 1 \times 1
\end{bmatrix}
=
\begin{bmatrix}
1 & -1 & 1 \\
-1 & 1 & -1 \\
1 & -1 & 1
\end{bmatrix}
$$

---

### 4. **ホップフィールドネットワークでの利用**
ホップフィールドネットワークでは、各パターンについてこの外積を計算し、それを **重み行列に累積** していきます。これにより、複数のパターンを同時に記憶することが可能になります。

#### 重み行列の更新
```python
self.weights += np.dot(p, p.T)
```

- ここで、`p` は列ベクトル（縦ベクトル）に変換されたパターンです。
- `np.dot(p, p.T)` を使って **外積** を計算し、重み行列 `self.weights` に加算します。
- この処理を **すべてのパターン** について繰り返すことで、重み行列が更新されます。

---

### 5. **重み行列が学習に与える影響**
重み行列は、ホップフィールドネットワークがパターンを記憶し、再現するための「記憶装置」として機能します。

- ノイズの入ったパターンを入力した際、重み行列は入力パターンを元に、記憶しているパターンに**収束**させる力を発揮します。
- これは、重み行列が各ニューロン間の結合を適切に調整しているためです。

---

### 6. **まとめ**
- **外積** は、ホップフィールドネットワークにおいて、パターンを記憶するための重み行列を構築するために利用されています。
- **重み行列** の各要素は、ニューロン同士の結合の強さを表し、ネットワークが入力パターンを正しく再現するために重要な役割を果たします。
- **外積** を利用することで、パターン同士の相関関係を重み行列に反映させ、パターンの連想記憶を実現しています。

---

この説明により、ホップフィールドネットワークの **外積** を利用した学習プロセスや、ノイズの入ったパターンから記憶したパターンを再現する仕組みがより深く理解できるようになるはずです。